# Traffic Analysis
<i>Adapted for use with FABRIC from [Traffic Analysis](https://www.cs.unc.edu/Research/geni/geniEdu/05-TrafficAnalysis.html)
    
In this tutorial you will learn a variety of tools for traffic analysis, with a focus on ping and tcpdump.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
    
This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Design the Experiment
### 1.1 Reserve Resources


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
slice_name = 'TrafficAnalysis'
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

### 1.2 Configure the Nodes

In [ ]:
# ADDDED Subnet
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.1.1.0/24")
available_ips = list(subnet)[1:]


network_name = 'lan'

In [ ]:
# configure nodes
try:
    for node in slice.get_nodes():
        node_iface = node.get_interface(network_name=network_name)  
        node_addr = available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=subnet)

        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

except Exception as e:
    print(f"Fail: {e}")

### 1.3 Installation
We will be installing net-tools package on both the server and client nodes, containing many useful networking tools.

In [ ]:
for node in slice.get_nodes():
    node.execute("sudo apt install net-tools")
    stdout, stderr = node.execute(f'sudo ifconfig ens7 up')

## 2. Experiment
### 2.1 ifconfig
1. On the client run:
<br>```ifconfig```
<br> This command lists the interfaces and their status. The interface facing the server is 'ens7' with the IP address of '10.1.1.2'.


### 2.2 Adjust the MTU
Maximum Transmission Unit (MTU) is a value representing the maximum length data packet that a a device (NIC) can accept.
1. On the server SSH terminal run:
<br>```ifconfig```

2. The MTU size can be adjusted with the following command, where 'interface name' corresponds to the interface facing the client:
<br>```sudo ifconfig <interface name> mtu 1400```
3. Repeat steps 1 and 2 on the client terminal.


### 2.3 Ping
Ping is one of the most popular networking tools, inheriting the name from sonar technology. The ping tool sends packets, and gathers information from their "echos". Such information includes determining if  a host is reachable and the round-trip time (RTT) to that host.
1. On the client terminal run:
<br>```ping 10.1.1.2```
<br>ICMP echo request packets are sent to the target host, and the host will generate ICMP responses. With the response the originating host calculates the RTT and detects lost packets.

2. Press Ctrl + C to stop pinging.
3. Ping can be verified by bringing down the connection, to do so run on the interface connected to the server (ens7):
<br>```sudo ip link set dev <interface name> down```
4. Ping the server again:
<br>```ping 10.1.1.2```
<br>Take note of the change in RTT
5. Bring the connection back up by running:
<br>```sudo ip link set dev <interface name> up```
6. Ping the server again:
<br>```ping 10.1.1.2```

### 2.4 Tcpdump
1. On the client terminal run:
<br>```sudo tcpdump -i <interface name> -nn```
<br>Where ```-i <interface name>``` is the previously used interface used to connect to the server node and -nn causes tcpdump to use the numerical IP addresses rather than using DNS to resolve the symbolic addresses. 
   
2. On the server terminal run:
<br>```ping 10.1.1.1```

3. Return to the client terminal and notice incoming packets looking similar to:
<br>```IP 10.1.1.2 > 10.1.1.1: ICMP echo request, id 2, seq 1, length 64```
<br>Each line corresponds to a packet travelling over the interface. The columns in the output correspond to:
    * Timestamp
    * Source IP
    * Destination IP
    * Layer 3 protocol name and associated information
4. Press ctrl + C on the client and server terminals to stop pinging and tcpdump.
5. On the client terminal run:
<br>```sudo tcpdump -i <interface name> -nn -S tcp```
<br> The addition of ```-S``` instructs tcpdump to print the absolute sequence numbers instead of the smaller, more human readable version. ```tcp``` filters the traffic to only show TCP data.
6. In the server terminal run:
<br>```nc -l 44444```
<br> This command has the netcat tool listen on port ```44444```. Netcat is a tool allowing for reading and writing both TCP and UDP traffic across a network.
7. Open an additional terminal on the client and run:
<br>```nc 10.1.1.1 44444```
<br>On the first client terminal (running tcpdump) you can see messages similar to:
<br>```10:26:33.508856 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [S], seq 1812506012, win 14600, options [mss 1460,sackOK,TS val 1375289021 ecr 0,nop,wscale 7], length 0```
<br>```10:26:33.508939 IP 10.1.1.2.44444 > 10.1.1.1.46520: Flags [S.], seq 3211754307, ack 1812506013, win 14480, options [mss 1460,sackOK,TS val 1375289021 ecr 1375289021,nop,wscale 7], length 0```
<br>```10:26:33.508958 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [.], ack 3211754308, win 115, options [nop,nop,TS val 1375289021 ecr 1375289021], length 0```
<br>These lines show the 3-way TCP handshake, initializing the connection between the server and the client. 
8. On the second client terminal (not running tcpdump) type:
<br>```Hello World```
<br>The netcat tool creates a TCP packet with this message and sends it to the server node.
<br>On the client terminal running tcpdump you should see messages similar to:
<br>```10:28:13.538448 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [P.], seq 1812506013:1812506025, ack 3211754308, win 115, options [nop,nop,TS val 1375389051 ecr 1375289021], length 12```
<br>```10:28:13.538530 IP 10.1.1.2.44444 > 10.1.1.1.46520: Flags [.], ack 1812506025, win 114, options [nop,nop,TS val 1375389051 ecr 1375389051], length 0```
<br> The first packet is the outgoing ```Hello World``` message, while the second packet is the acknowledgement (ACK).
9. On the server terminal type:
<br>```Hello World```
<br>On the client terminal running tcpdump, messages similar to the previous step should be present and sent in the opposite direction.
10. Press ctrl + C on all terminals to close out of the running programs.

### 2.5 More on Tcpdump Options
1. tcpdump is able to write what it captures to disk rather than just displaying it on the screen. However, care should be taken to honor privacy when using tcpdump. It is customary to only capture a certain number of bytes of each packet's header. This is done by setting a snap length of N. Only the data in the first N bytes is recorded by tcpdump. Restart tcpdump in the client SSH terminal, but use the "-w" option to write to a file and add a snaplength of 96 bytes with the "-s" option:
```sudo tcpdump -i <interface name> -nn -S -s 96 -w <filename.pcap> tcp```
2. In the server SSH terminal, start the iperf server by typing:
```iperf -s```
3. in the client ssh terminal, start the iperf client by typing 
```iperf -c <server_ip> -t 1```
4. After 1 second the client should have completed sending its traffic. ```Type Ctrl-c``` in the tcpdump window and in the iperf server.
5. tcpdump can display and filter preeviously captured ppcapp files. To display the first 10 packets use the "-c" and the read "r" options: ```tcpdump -nn -r <filename.pcap> -c 10S```
5. Now, display only packets whose source is the server (with IP address 10.1.1.2). To do so, we add an "and" and a "src" attribute to the filter string, like this:
```tcpdump -nn -r <filename.pcap> -c 10 tcp and src 10.1.1.2```

## Continue to The Assignment notebook

Once You have completed this notebook you should be able to continue to the Traffic Analysis Assignment Notebook, you can ether open it on the explorer or click [Here](./Assignment.ipynb) to open the next notebook.